## Inheritance

In [26]:
class Card():
    """Playing card"""
    
    rankMap = [("%d" % i) for i in range(13)] # Class attributes
    (rankMap[0],rankMap[1],*rankMap[11:13]) = ('None','A','J','Q','K')
    # Interestingly, it seems that I can only use * in the left side once, so I cannot * both vectors
    # print(rankMap)
    suitMap = ['♣','♢','♡','♠']
    
    
    def __init__(self,suit=0,rank=1):
        self.suit = suit # Instance attributes
        self.rank = rank
        
    def __str__(self):
        return "%s%s" % (Card.rankMap[self.rank] , Card.suitMap[self.suit])
           
    def __eq__(self,other):
        return (self.rank==other.rank) & (self.suit==other.suit)
    
    def __gt__(self,other): 
        # They removed __cmp__ in Python 3, replacing it with: eq, ne, lt, le, gt, ge
        # However, Python somehow guesses how > will behave knowing <. So the minimum includes EQ, GT & GE
        # And then there's also __repr__ that is used for eval(). In this case, will be "Card(0,1)" - but with actual values
        return (self.suit, self.rank) > (other.suit, other.rank) # Uses the fancy lexicographicish way tuples are compared
        
    def __ge__(self,other):
        return self.__gt__(other) | self.__eq__(other)
        

a = Card(0,2)
print(a)
print(a==Card(0,3))
print(a!=Card(0,3))
print(a>Card(0,3))
print(a<Card(0,3))
print(a<=Card(0,3))
print(a>=Card(0,3))

2♣
False
True
False
True
True
False


In [22]:
import random

In [53]:
class Deck(object):
    """Deck of cards"""
    
    def __init__(self):
        self.cards = []
        self.label = 'Deck'
        for s in range(4):
            for r in range(1,14):
                self.cards.append(Card(s,r))
                
    def __str__(self):
        res = []
        res.append('%s[%d]:' % (self.label,len(self.cards)))
        for card in self.cards:
            # res = res+str(card)+'|' # This is slow, apparently
            res.append(str(card)) # Fast: first a list, then join all with a delim
        return ' '.join(res)
    
    def pop(self): # Returns last card and updates the array (actual pop)
        return self.cards.pop()
    
    def add(self,card):
        self.cards.append(card)
        
    def shuffle(self):
        random.shuffle(self.cards)
        
    def deal(self,hand,num=1):
        for i in range(num):
            hand.add(self.pop())
    
                
d = Deck()
print(d)
print(d.pop())
d.add(Card(2,2))
print(d)
d.shuffle()
print(d)

Deck[52]: A♣ 2♣ 3♣ 4♣ 5♣ 6♣ 7♣ 8♣ 9♣ 10♣ J♣ Q♣ K♣ A♢ 2♢ 3♢ 4♢ 5♢ 6♢ 7♢ 8♢ 9♢ 10♢ J♢ Q♢ K♢ A♡ 2♡ 3♡ 4♡ 5♡ 6♡ 7♡ 8♡ 9♡ 10♡ J♡ Q♡ K♡ A♠ 2♠ 3♠ 4♠ 5♠ 6♠ 7♠ 8♠ 9♠ 10♠ J♠ Q♠ K♠
K♠
Deck[52]: A♣ 2♣ 3♣ 4♣ 5♣ 6♣ 7♣ 8♣ 9♣ 10♣ J♣ Q♣ K♣ A♢ 2♢ 3♢ 4♢ 5♢ 6♢ 7♢ 8♢ 9♢ 10♢ J♢ Q♢ K♢ A♡ 2♡ 3♡ 4♡ 5♡ 6♡ 7♡ 8♡ 9♡ 10♡ J♡ Q♡ K♡ A♠ 2♠ 3♠ 4♠ 5♠ 6♠ 7♠ 8♠ 9♠ 10♠ J♠ Q♠ 2♡
Deck[52]: 9♠ 2♢ 7♠ 9♢ Q♡ 3♡ 6♣ 5♡ K♡ 10♡ 2♡ 7♡ 9♡ A♢ 2♡ 2♠ K♣ 10♣ Q♣ 5♣ J♢ A♠ 7♣ 8♣ 3♢ 4♡ 6♠ 7♢ 6♡ J♠ 10♠ Q♠ 4♢ Q♢ 5♢ 9♣ J♣ 8♢ A♣ 3♣ 3♠ J♡ K♢ 8♠ 8♡ 4♠ A♡ 2♣ 6♢ 4♣ 5♠ 10♢


In [69]:
def find_defining_class(obj,_method): # Author's advice on localizing which class a method is inhereted from
    for _type in type(obj).mro(): # mro() stands for Method Resolution Order
        if _method in _type.__dict__:
            return _type

find_defining_class(h, 'shuffle')

__main__.Deck

In [207]:
class Hand(Deck): # Inheritance: inherits to the main Deck class
    """Hand of cards"""
    
    def __init__(self,label=''):
        self.cards = []
        self.label = label
        
    def take(self,deck,n=1):
        deck.deal(self,n)
        
    def classify(self):
        """Classifies a poker hand"""
        self.label = 'none'
        self.cards.sort() # Because we defined all <>, we can sort cards! May be helpful.
        histR = {}
        histS = {}
        for c in self.cards:
            histR[c.rank] = histR.get(c.rank,0)+1
            histS[c.suit] = histS.get(c.suit,0)+1
        v = [histR[_] for _ in histR.keys()]
        # Straight flush should be here
        if max(v)==4:
            self.label = 'Four'
        elif sum([_==3 for _ in v])==1 & sum([_==2 for _ in v])==1:
            self.label = 'House'
        elif max([histS[_] for _ in histS.keys()])==5:
            self.label = 'Flush'
        # Check for straight should be here
        elif max(v)==3:
            self.label = 'Three'
        elif sum([_==2 for _ in v])==2:
            self.label = '2 pairs'
        elif sum([_==2 for _ in v])==1:
            self.label = 'Pair'
        
        
for j in range(5):
    d = Deck()
    d.shuffle()
    for i in range(5):
        h = Hand()
        h.take(d,7)
        h.classify()
        print(h)

none[7]: 8♣ Q♢ K♢ 10♡ J♡ 2♠ 7♠
none[7]: A♣ 10♣ A♢ 7♢ 10♢ K♡ K♠
none[7]: 7♣ J♣ Q♣ 2♢ 6♡ 5♠ 10♠
Pair[7]: 9♣ 4♢ 9♢ 3♡ 5♡ 8♡ J♠
Three[7]: 6♣ 6♢ 9♡ Q♡ 3♠ 6♠ 8♠
2 pairs[7]: 2♣ 7♣ J♣ 10♢ 7♡ K♡ 10♠
none[7]: 8♣ 2♢ 3♢ 6♡ J♡ 4♠ 7♠
Pair[7]: 6♢ 9♢ K♢ 9♡ Q♡ 2♠ 5♠
none[7]: 3♣ J♢ Q♢ A♡ 2♡ 8♡ 6♠
Pair[7]: 5♣ 6♣ 5♢ 4♡ A♠ 3♠ 8♠
Pair[7]: 7♣ 3♢ 8♢ 9♡ 10♡ A♠ 9♠
2 pairs[7]: 5♣ Q♣ 7♡ K♡ 6♠ 7♠ Q♠
none[7]: 4♣ 9♣ K♣ 6♢ 5♡ 3♠ 10♠
Pair[7]: A♣ 2♣ 10♢ Q♢ A♡ 8♡ K♠
2 pairs[7]: 6♣ 8♣ 4♢ 5♢ 2♠ 4♠ 5♠
2 pairs[7]: 2♣ K♣ K♢ A♡ 8♡ 9♡ 8♠
Pair[7]: 5♣ 8♢ 5♡ J♡ 9♠ 10♠ K♠
2 pairs[7]: 3♣ 6♣ 2♢ 6♢ 4♡ A♠ 4♠
Pair[7]: 4♣ 2♡ Q♡ 3♠ 5♠ 6♠ Q♠
none[7]: 7♣ 8♣ Q♣ 4♢ 5♢ 10♢ J♠
none[7]: 3♣ 8♣ 6♢ J♢ A♡ 5♡ 2♠
none[7]: 4♣ 9♣ J♣ 5♢ 8♡ 10♠ K♠
Three[7]: Q♣ A♢ 7♢ Q♡ 9♠ J♠ Q♠
none[7]: 7♣ Q♢ K♢ 3♡ 6♡ 10♡ A♠
none[7]: 6♣ 2♢ 8♢ 9♢ K♡ 3♠ 7♠
